# Data Preprocessing

### beer_reviews.csv (https://www.kaggle.com/datasets/rdoume/beerreviews)

- 환경
  - Python 3.9.7
  - numpy 1.21.5
  - pandas 1.4.1

- 파일구조
  ```text
  project
     |
     |___ beer_reviews
     |         |
     |         |___ beer_reviews.csv
     |
     |___ data_preprocessing.ipynb
  ```
- 이 파일은 beer_reviews에 새로운 csv를 생성합니다. (맨 아래 셀 참조)

In [51]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

### 데이터 살피기
데이터의 구조를 파악하고 어떤 값이 들어있는지 살핀다.

review데이터의 첫번째부터 5개의 값을 확인한다.

In [22]:
reviews.head(5)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


|Column_Name		|Meaning										|
|:------------------|:----------------------------------------------|
|brewery_id			|양조장 식별번호								|
|review_time		|리뷰를 한 시간, 타임스탬프						|
|review_overall		|전체적인 리뷰 평점 평균						|
|review_aroma		|맥주의 향에 대한 리뷰 평점						|
|review_appearance	|맥주의 보이는 부분(색, 형태 등)에 대한 리뷰 평점|
|review_profilename	|리뷰 평점을 남긴 사용자						|
|beer_style			|맥주의 종류									|
|review_palate		|맥주의 목넘김에 대한 리뷰 평점					|
|review_taste		|맥주의 맛에 대한 리뷰 평점						|
|beer_name			|맥주의 이름									|
|beer_abv			|맥주의 도수									|
|beer_beerid		|맥주 식별번호									|

각 열의 의미를 알 수 있었다.

review데이터의 뒤에서부터 5개의 값을 확인한다.

In [23]:
reviews.tail(5)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
1586609,14359,The Defiant Brewing Company,1162684892,5.0,4.0,3.5,maddogruss,Pumpkin Ale,4.0,4.0,The Horseman's Ale,5.2,33061
1586610,14359,The Defiant Brewing Company,1161048566,4.0,5.0,2.5,yelterdow,Pumpkin Ale,2.0,4.0,The Horseman's Ale,5.2,33061
1586611,14359,The Defiant Brewing Company,1160702513,4.5,3.5,3.0,TongoRad,Pumpkin Ale,3.5,4.0,The Horseman's Ale,5.2,33061
1586612,14359,The Defiant Brewing Company,1160023044,4.0,4.5,4.5,dherling,Pumpkin Ale,4.5,4.5,The Horseman's Ale,5.2,33061
1586613,14359,The Defiant Brewing Company,1160005319,5.0,4.5,4.5,cbl2,Pumpkin Ale,4.5,4.5,The Horseman's Ale,5.2,33061


각 컬럼에 어떤 값이 들어가는지 파악할 수 있다. 

### 데이터 전처리
reviews 데이터프레임의 기술 통계량을 확인한다.

In [24]:
reviews.describe()

,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid
count,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.518829e+06,1.586614e+06
mean,3.130099e+03,1.224089e+09,3.815581e+00,3.735636e+00,3.841642e+00,3.743701e+00,3.792860e+00,7.042387e+00,2.171279e+04
std,5.578104e+03,7.654427e+07,7.206219e-01,6.976167e-01,6.160928e-01,6.822184e-01,7.319696e-01,2.322526e+00,2.181834e+04
min,1.000000e+00,8.406720e+08,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02,3.000000e+00
25%,1.430000e+02,1.173224e+09,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,5.200000e+00,1.717000e+03
50%,4.290000e+02,1.239203e+09,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,6.500000e+00,1.390600e+04
75%,2.372000e+03,1.288568e+09,4.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,8.500000e+00,3.944100e+04
max,2.800300e+04,1.326285e+09,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.770000e+01,7.731700e+04


<span style="color: red">brewery_id, beer_beerid, review_time column은 수치형 자료가 아닌 범주형 자료 이므로, 기술통계량이 의미가 없다.<span>

<span style="color: blue">brewery_id와 beer_beerid, review_time column을 범주형 자료로 바꾼다.<span>

In [25]:
reviews["brewery_id"] = reviews["brewery_id"].astype('category')
reviews["beer_beerid"] = reviews["beer_beerid"].astype('category')
reviews["review_time"] = reviews["review_time"].astype('category')
reviews.describe()

,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv
count,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.518829e+06
mean,3.815581e+00,3.735636e+00,3.841642e+00,3.743701e+00,3.792860e+00,7.042387e+00
std,7.206219e-01,6.976167e-01,6.160928e-01,6.822184e-01,7.319696e-01,2.322526e+00
min,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02
25%,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,5.200000e+00
50%,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,6.500000e+00
75%,4.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,8.500000e+00
max,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.770000e+01


각 Column의 최대값, 최솟값, 중앙값과 분포도를 대충 파악할 수 있다.

reviews 데이터 프레임의 정보를 확인한다.

In [26]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype   
---  ------              --------------    -----   
 0   brewery_id          1586614 non-null  category
 1   brewery_name        1586599 non-null  object  
 2   review_time         1586614 non-null  category
 3   review_overall      1586614 non-null  float64 
 4   review_aroma        1586614 non-null  float64 
 5   review_appearance   1586614 non-null  float64 
 6   review_profilename  1586266 non-null  object  
 7   beer_style          1586614 non-null  object  
 8   review_palate       1586614 non-null  float64 
 9   review_taste        1586614 non-null  float64 
 10  beer_name           1586614 non-null  object  
 11  beer_abv            1518829 non-null  float64 
 12  beer_beerid         1586614 non-null  category
dtypes: category(3), float64(6), object(4)
memory usage: 183.2+ MB


<span style="color:red">null값을 갖는 데이터가 있다.<span>

<span style="color:blue">null값을 갖는 데이터 갯수를 확인하고,<span>

In [27]:
print(reviews.isnull().sum())

brewery_id                0
brewery_name             15
review_time               0
review_overall            0
review_aroma              0
review_appearance         0
review_profilename      348
beer_style                0
review_palate             0
review_taste              0
beer_name                 0
beer_abv              67785
beer_beerid               0
dtype: int64


<span style="color: blue">null값을 갖는 데이터를 제거한다.<span>

In [28]:
reviews = reviews.dropna()
print(reviews.isnull().sum())

brewery_id            0
brewery_name          0
review_time           0
review_overall        0
review_aroma          0
review_appearance     0
review_profilename    0
beer_style            0
review_palate         0
review_taste          0
beer_name             0
beer_abv              0
beer_beerid           0
dtype: int64


In [29]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1518478 entries, 0 to 1586613
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype   
---  ------              --------------    -----   
 0   brewery_id          1518478 non-null  category
 1   brewery_name        1518478 non-null  object  
 2   review_time         1518478 non-null  category
 3   review_overall      1518478 non-null  float64 
 4   review_aroma        1518478 non-null  float64 
 5   review_appearance   1518478 non-null  float64 
 6   review_profilename  1518478 non-null  object  
 7   beer_style          1518478 non-null  object  
 8   review_palate       1518478 non-null  float64 
 9   review_taste        1518478 non-null  float64 
 10  beer_name           1518478 non-null  object  
 11  beer_abv            1518478 non-null  float64 
 12  beer_beerid         1518478 non-null  category
dtypes: category(3), float64(6), object(4)
memory usage: 188.9+ MB


맥주에 대한 평가만을 갖는 dataframe을 만든다. 맥주와 리뷰어에 대한 Utility Matrix를 만들기 위해 필요 없는 Column을 배제한다.

만들어진 dataframe을 review_overall 값에 대하여 내림차순으로 정렬한다.

In [30]:
beer_review = reviews[["beer_name","beer_beerid", "review_profilename", "review_aroma", "review_appearance", "review_palate", "review_taste", "review_overall"]]
print(beer_review.shape)

beer_review = beer_review.sort_values("review_overall", ascending=False)
beer_review.reset_index(drop=True, inplace=True)
beer_review.head(10)

(1518478, 8)


,beer_name,beer_beerid,review_profilename,review_aroma,review_appearance,review_palate,review_taste,review_overall
0,The Horseman's Ale,33061,cbl2,4.5,4.5,4.5,4.5,5.0
1,Wittekerke,2477,BEERchitect,4.0,4.5,4.5,4.0,5.0
2,Moe's Bender,58120,deadbody,5.0,5.0,5.0,5.0,5.0
3,Moe's Bender,58120,pdeme,5.0,4.0,4.5,5.0,5.0
4,World Wide Stout,2392,Sayson,4.5,4.0,4.5,4.0,5.0
5,World Wide Stout,2392,Mauhyr,5.0,4.5,4.5,5.0,5.0
6,World Wide Stout,2392,Jacobob10,4.0,4.0,4.5,5.0,5.0
7,World Wide Stout,2392,AlexJ,4.5,4.5,5.0,5.0,5.0
8,World Wide Stout,2392,Nutleybeer,4.0,4.0,5.0,4.5,5.0
9,Hell,44531,Ryan011235,4.5,4.0,4.0,4.5,5.0


맥주이름과 리뷰어가 같은 데이터는 중복이므로 제거한다.

이 때, 평점이 더 높은 값을 취하고 낮은 값을 제거한다.

In [31]:
beer_review = beer_review.drop_duplicates(subset=["beer_name", "review_profilename"], keep="first")
print(beer_review.shape)
beer_review.info()

(1496263, 8)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1496263 entries, 0 to 1518477
Data columns (total 8 columns):
 #   Column              Non-Null Count    Dtype   
---  ------              --------------    -----   
 0   beer_name           1496263 non-null  object  
 1   beer_beerid         1496263 non-null  category
 2   review_profilename  1496263 non-null  object  
 3   review_aroma        1496263 non-null  float64 
 4   review_appearance   1496263 non-null  float64 
 5   review_palate       1496263 non-null  float64 
 6   review_taste        1496263 non-null  float64 
 7   review_overall      1496263 non-null  float64 
dtypes: category(1), float64(5), object(2)
memory usage: 99.6+ MB


Utility Matrix를 만들어 사용하기 위해서는 평가를 하지 않은 것과 평점이 0점인것에 대한 구분을 해주어야한다.

따라서, 평점이 0점인 것들을 모두 삭제한다. 앞으로 0은 평가를 하지 않은 것을 의미한다.
앞서 기술통계량을 살펴보았을 때 평점을 나타내는 Column에서 review_overall과 review_appearance만 0값을 가졌으므로 두 컬럼만 처리한다.

In [32]:
beer_review_nonspar = beer_review[(beer_review['review_overall'] >=1)|(beer_review['review_appearance'] >=1)]
print(beer_review_nonspar.shape)
beer_review_nonspar.describe()

(1496256, 8)


,review_aroma,review_appearance,review_palate,review_taste,review_overall
count,1.496256e+06,1.496256e+06,1.496256e+06,1.496256e+06,1.496256e+06
mean,3.746420e+00,3.850323e+00,3.754199e+00,3.804605e+00,3.825561e+00
std,6.953686e-01,6.143585e-01,6.793359e-01,7.287219e-01,7.175819e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
25%,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00
50%,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00
75%,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,4.500000e+00
max,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00


평점의 최솟값이 모두 1임을 알 수 있다.

전처리된 dataframe을 새로운 csv 파일로 만들자.

In [33]:
# beer_review_nonspar.reset_index(drop=True, inplace=False)
# beer_review_nonspar.to_csv("./beer_review_for_utility_matrix.csv")

In [34]:
beer_review_nonspar.head(5)

,beer_name,beer_beerid,review_profilename,review_aroma,review_appearance,review_palate,review_taste,review_overall
0,The Horseman's Ale,33061,cbl2,4.5,4.5,4.5,4.5,5.0
1,Wittekerke,2477,BEERchitect,4.0,4.5,4.5,4.0,5.0
2,Moe's Bender,58120,deadbody,5.0,5.0,5.0,5.0,5.0
3,Moe's Bender,58120,pdeme,5.0,4.0,4.5,5.0,5.0
4,World Wide Stout,2392,Sayson,4.5,4.0,4.5,4.0,5.0


In [82]:
beer_review_nonspar.tail(5)

,beer_name,beer_beerid,review_profilename,review_aroma,review_appearance,review_palate,review_taste,review_overall
1518466,Busch Beer,670,Kamuimusackie,1.5,1.5,1.0,1.5,1.0
1518467,The Kaiser,19425,Jdiddy,1.0,3.0,1.0,1.0,1.0
1518468,Honey's,59818,Lakes,2.0,3.5,2.0,2.0,1.0
1518469,De Zwarte Ruiter,10685,francisweizen,2.0,2.5,1.5,1.5,1.0
1518470,Bud Light,1320,rhoadsrage,2.0,1.0,1.5,2.0,1.0


### Collaborative Filtering
유저아이디가 100개 이상 존재하고, 하나의 맥주가 100개 이상의 리뷰를 가지고 있으면 남겨놓는다.

In [66]:
beer_user = beer_review_nonspar.copy()
min_id = beer_user['review_profilename'].value_counts() >= 100
min_id = min_id[min_id].index.to_list()
print(len(min_id))
beer_user = beer_user[beer_user['review_profilename'].isin(min_id)]

min_beer = beer_user['beer_name'].value_counts() >= 100
min_beer = min_beer[min_beer].index.to_list()
beer_user = beer_user[beer_user['beer_name'].isin(min_beer)]
print(len(min_beer))
print(beer_user.head(5))
print(beer_user.info())
print('------------------------')
print(beer_user.count())
print(beer_user[beer_user['beer_name']=='# 100'])
print(beer_user[beer_user['review_profilename']=='0110x011'])

3083
2697
           beer_name beer_beerid review_profilename  review_aroma  \
1         Wittekerke        2477        BEERchitect           4.0   
6   World Wide Stout        2392          Jacobob10           4.0   
7   World Wide Stout        2392              AlexJ           4.5   
9               Hell       44531         Ryan011235           4.5   
13  World Wide Stout        2392         Litterbean           4.0   

    review_appearance  review_palate  review_taste  review_overall  
1                 4.5            4.5           4.0             5.0  
6                 4.0            4.5           5.0             5.0  
7                 4.5            5.0           5.0             5.0  
9                 4.0            4.0           4.5             5.0  
13                4.0            4.0           4.5             5.0  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 831348 entries, 1 to 1518470
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype   

맥주 x 유저 매트릭스 생성

In [67]:
beer_user_rating = beer_user.pivot_table('review_overall', index='beer_name',columns='review_profilename').fillna(0)

In [68]:
beer_user_rating.shape

(2697, 3083)

In [69]:
beer_user_rating.head(10)

review_profilename,0110x011,05Harley,100floods,1759Girl,1Adam12,1fastz28,321jeff,3Vandoo,4000qtrap,4DAloveofSTOUT,...,zeledonia,zerk,zerofear13,zhanson88,zimm421,zjungleist,zoolzoo,zoso1967,zoso493,zrab11
beer_name,,,,,,,,,,,,,,,,,,,,,
"""Shabadoo"" Black & Tan Ale",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
# 100,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,4.0,4.0,0.0,3.5,0.0,0.0,0.0,4.0,0.0
10 Commandments,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000 IBU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
10th Anniversary Double India Pale Ale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120 Minute IPA,0.0,4.0,0.0,0.0,0.0,3.0,0.0,4.5,0.0,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,1.5,3.5,0.0,0.0
12th Anniversary Undercover Investigation Shut-Down Ale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14'ER ESB,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


코싸인 유사도 구하기

In [75]:
item_sim = cosine_similarity(beer_user_rating)
print(item_sim)

[[1.         0.0639323  0.20005384 ... 0.09126609 0.08121036 0.10520632]
 [0.0639323  1.         0.18240455 ... 0.29459474 0.14918519 0.24379373]
 [0.20005384 0.18240455 1.         ... 0.24457948 0.24432119 0.27368947]
 ...
 [0.09126609 0.29459474 0.24457948 ... 1.         0.1868337  0.31017872]
 [0.08121036 0.14918519 0.24432119 ... 0.1868337  1.         0.22339465]
 [0.10520632 0.24379373 0.27368947 ... 0.31017872 0.22339465 1.        ]]


계산된 코싸인 유사도 데이터 프레임으로 생성

In [76]:
item_based_col = pd.DataFrame(data = item_sim, index = beer_user_rating.index, columns = beer_user_rating.index)

In [77]:
item_based_col.head()

beer_name,"""Shabadoo"" Black & Tan Ale",# 100,#9,10 Commandments,1000 IBU,10th Anniversary Double India Pale Ale,120 Minute IPA,12th Anniversary Undercover Investigation Shut-Down Ale,14'ER ESB,1554 Enlightened Black Ale,...,ZÔN,Žatec,§ucaba (Abacus),Édition 2005,Éphémère (Apple),Éphémère (Cassis / Black Currant),Équinoxe Du Printemps,Ølfabrikken Porter,Über Pils,ÜberSun (Imperial Summer Wheat Beer)
beer_name,,,,,,,,,,,,,,,,,,,,,
"""Shabadoo"" Black & Tan Ale",1.000000,0.063932,0.200054,0.087172,0.032573,0.136888,0.152117,0.109222,0.092600,0.111402,...,0.057253,0.061859,0.052401,0.083686,0.157676,0.092423,0.054988,0.091266,0.081210,0.105206
# 100,0.063932,1.000000,0.182405,0.236280,0.249178,0.169692,0.252696,0.176943,0.167968,0.207777,...,0.139024,0.140491,0.188929,0.157257,0.206489,0.121108,0.253785,0.294595,0.149185,0.243794
#9,0.200054,0.182405,1.000000,0.187536,0.111747,0.238663,0.410182,0.180748,0.274392,0.381473,...,0.192303,0.158672,0.132307,0.212298,0.417340,0.236716,0.158109,0.244579,0.244321,0.273689
10 Commandments,0.087172,0.236280,0.187536,1.000000,0.154828,0.158045,0.259582,0.182424,0.152828,0.242954,...,0.173766,0.113163,0.285389,0.146014,0.206482,0.114382,0.198566,0.340327,0.108792,0.246454
1000 IBU,0.032573,0.249178,0.111747,0.154828,1.000000,0.122884,0.195021,0.067756,0.087986,0.112075,...,0.099430,0.098181,0.236539,0.069757,0.100795,0.050069,0.163127,0.221316,0.068678,0.139492


코싸인 유사도가 높은 5개의 값 뽑기

In [79]:
item_based_col['World Wide Stout'].sort_values(ascending=False)[:5]

beer_name
World Wide Stout                1.000000
90 Minute IPA                   0.568650
120 Minute IPA                  0.564272
Stone Imperial Russian Stout    0.562352
Storm King Stout                0.560923
Name: World Wide Stout, dtype: float64

In [80]:
item_based_col['Hell'].sort_values(ascending=False)[:5]

beer_name
Hell             1.000000
Bitter Brewer    0.518241
Wet              0.499873
CynicAle         0.461807
Abrasive Ale     0.456073
Name: Hell, dtype: float64

In [83]:
item_based_col['Bud Light'].sort_values(ascending=False)[:5]

beer_name
Bud Light                    1.000000
Coors Light                  0.545511
Budweiser                    0.544985
Miller Lite                  0.512699
Samuel Adams Boston Lager    0.488824
Name: Bud Light, dtype: float64

In [87]:
item_based_col['Busch Beer'].sort_values(ascending=False)[:5]

beer_name
Busch Beer     1.000000
Bud Ice        0.459782
Icehouse       0.436814
Coors          0.436744
Natural Ice    0.419480
Name: Busch Beer, dtype: float64